### 네거티브 샘플링을 이용한 Word2Vec 구현(Skip-Gram with Negative Sampling, SGNS)

- Word2Vec 출력층에서는 소프트맥스 함수로부터 결과값을 임베딩 벡터값을 업데이트한다.
    - 학습이 무거운 모델이다
- 네거티브 샘플링은 일부 단어 집합에만 집중시키는 방법이다.
    - 고양이 , 귀여운 에만 집중
    - 훨씬 작은 단어 집합을 만ㄷ르어 마지막 단계를 이진 분류 문제로 변환한다.
    - 주변 단어를 긍정, 랜덤으로 샘플링 단어를 부정으로 레이블링 ( 이진 분류 문제 데이터셋)
    

Skip-gram 은 주변 단어로부터 중심단어를 예측한다.

![](https://wikidocs.net/images/page/69141/%EA%B7%B8%EB%A6%BC1.PNG)

![](https://wikidocs.net/images/page/69141/%EA%B7%B8%EB%A6%BC1-1.PNG)

네커티브 샘플링의 경우, Skip-gram with Negative Sampling, SGNS)
- 중심 단어, 주변 단어가 모두 입력이 된다
- 그 후 실제로 윈도우 크기 내에 존재하는 이웃 관계인지 그 확률을 예측한다.

![](https://wikidocs.net/images/page/69141/%EA%B7%B8%EB%A6%BC1-2.PNG)

- 좌측의 테이블 : Skip-gram 학습 데이터셋
- 우측의 테이블 : SGNS 학습
    - 기존의 Skip-gram 데이터셋에서 중심단어와 주변단어를 각각 입력1, 입력2로 둔다
    - 이웃관계는 레이블을 1로 한다. 나머지 0


![](https://wikidocs.net/images/page/69141/%EA%B7%B8%EB%A6%BC3.PNG)

- 2 개의 임베딩 테이블을 준비한다
    - 테이블 중 하나는 입력 1인 중심 단어의 테이블 룩업을 위한 임베딩 테이블
    - 다른 하나는 입력 2인 주변 단어의 테이블 룩업을 위한 임베딩 테이블이다
![](https://wikidocs.net/images/page/69141/%EA%B7%B8%EB%A6%BC5.PNG)

![](https://wikidocs.net/images/page/69141/%EA%B7%B8%EB%A6%BC7.PNG)

#### 20 뉴스 그룹 데이터 전처리하기

In [1]:
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from sklearn.datasets import fetch_20newsgroups
from tensorflow.keras.preprocessing.text import Tokenizer

In [2]:
dataset = fetch_20newsgroups(shuffle=True, random_state=1, remove=('headers', 'footers', 'quotes'))
documents = dataset.data
print('총 샘플 수 :',len(documents))

총 샘플 수 : 11314


In [3]:
news_df = pd.DataFrame({'document':documents})
news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x: ' '.join([w for w in x.split() if len(w)>3]))
news_df['clean_doc'] = news_df['clean_doc'].apply(lambda x : x.lower())

C:\Users\CTLAB\AppData\Local\Temp/ipykernel_15024/3901852537.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  news_df['clean_doc'] = news_df['document'].str.replace("[^a-zA-Z]", " ")


In [4]:
news_df.replace("", float("NaN"), inplace=True)
news_df.isnull().values.any()

True

In [5]:
news_df.dropna(inplace=True)
print("총 샘플 수 : ",len(news_df))

총 샘플 수 :  10995


불용어제거

In [7]:
stop_words = stopwords.words('english')
tokenized_doc = news_df['clean_doc'].apply(lambda x : x.split())
tokenized_doc = tokenized_doc.apply(lambda x : [item for item in x if item not in stop_words])
tokenized_doc = tokenized_doc.to_list()

In [9]:
# 단어 1개 이하인 샘플의 인덱스 찾아서 저장, 해당 샘플은 제거
drop_train = [index for index, sentence in enumerate(tokenized_doc) if len(sentence)<=1]
tokenized_doc = np.delete(tokenized_doc, drop_train, axis=0)
print("총 샘플 수 : ",len(tokenized_doc))

총 샘플 수 :  10940


In [10]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(tokenized_doc)

word2idx = tokenizer.word_index
idx2word = {value : key for key,value in word2idx.items()}

encoded = tokenizer.texts_to_sequences(tokenized_doc)

In [12]:
print(encoded[:1])

[[9, 59, 603, 207, 3278, 1495, 474, 702, 9470, 13686, 5533, 15227, 702, 442, 702, 70, 1148, 1095, 1036, 20294, 984, 705, 4294, 702, 217, 207, 1979, 15228, 13686, 4865, 4520, 87, 1530, 6, 52, 149, 581, 661, 4406, 4988, 4866, 1920, 755, 10668, 1102, 7837, 442, 957, 10669, 634, 51, 228, 2669, 4989, 178, 66, 222, 4521, 6066, 68, 4295]]


In [13]:
vocab_size = len(word2idx) + 1 
print('단어 집합의 크기 :', vocab_size)

단어 집합의 크기 : 64277


#### 4. 네거티브 샘플링을 통한 데이터셋 구성하기

In [14]:
from tensorflow.keras.preprocessing.sequence import skipgrams
# 상위 10개의 뉴스샘플에 대해서만 실행
skip_grams = [skipgrams(sample, vocabulary_size=vocab_size, window_size=10) for sample in encoded[:10]]


In [15]:
# 윈도우 크기 내에서 중심 단어, 주변 단어의 관계를 가지면 1
# 아니면 0


# 첫번쨰 뉴스그룹 샘플 확인하기
pairs, labels = skip_grams[0][0], skip_grams[0][1]
for i in range(5):
    print("({:s} ({:d}), {:s} ({:d})) -> {:d}".format(
          idx2word[pairs[i][0]], pairs[i][0], 
          idx2word[pairs[i][1]], pairs[i][1], 
          labels[i]))

(subsidizing (15228), least (87)) -> 1
(received (634), sillinger (61071)) -> 0
(whole (217), incidences (20294)) -> 1
(makes (228), hardy (11050)) -> 0
(lived (1148), unchecked (21976)) -> 0


In [16]:
print('전체 샘플 수 :',len(skip_grams))

전체 샘플 수 : 10


In [17]:
# 첫번째 뉴스그룹 샘플에 대해서 생긴 pairs와 labels의 개수
print(len(pairs))
print(len(labels))

2220
2220


In [18]:
skip_grams = [skipgrams(sample, vocabulary_size=vocab_size, window_size=10) for sample in encoded]

### 5. SGNS 구현하기

In [ ]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Embedding, Reshape, Activation, Input
from tensorflow.keras.layers import Dot
from tensorflow.keras.utils import plot_model
from IPython.display import SVG

In [ ]:
embedding_dim = 100

# 중심 단어를 위한 임베딩 테이블
w_inputs = Input(shape=(1, ), dtype='int32')
word_embedding = Embedding(vocab_size, embedding_dim)(w_inputs)

# 주변 단어를 위한 임베딩 테이블
c_inputs = Input(shape=(1, ), dtype='int32')
context_embedding  = Embedding(vocab_size, embedding_dim)(c_inputs)

dot_product = Dot(axes=2)([word_embedding, context_embedding])
dot_product = Reshape((1,), input_shape=(1, 1))(dot_product)
output = Activation('sigmoid')(dot_product)

model = Model(inputs=[w_inputs, c_inputs], outputs=output)
model.summary()
model.compile(loss='binary_crossentropy', optimizer='adam')
plot_model(model, to_file='model3.png', show_shapes=True, show_layer_names=True, rankdir='TB')